## Installing Packages

In [1]:
!pip install matplotlib
!pip install seaborn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


## Importing Libraries

In [2]:
import pandas as pd
import numpy as np
import re, json
import matplotlib.pyplot as plt
import seaborn as sns
import warnings as wn
wn.filterwarnings('ignore')

## Loading Dataset

In [3]:
df = pd.read_csv('The Ultimate Cars Dataset 2024.csv',encoding='cp1252')

In [4]:
df.head(5)

,Company Names,Cars Names,Engines,CC/Battery Capacity,HorsePower,Total Speed,Performance(0 - 100 )KM/H,Cars Prices,Fuel Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990 cc,963 hp,340 km/h,2.5 sec,"$1,100,000",plug in hyrbrid,2,800 Nm
1,ROLLS ROYCE,PHANTOM,V12,6749 cc,563 hp,250 km/h,5.3 sec,"$460,000",Petrol,5,900 Nm
2,Ford,KA+,1.2L Petrol,"1,200 cc",70-85 hp,165 km/h,10.5 sec,"$12,000-$15,000",Petrol,5,100 - 140 Nm
3,MERCEDES,GT 63 S,V8,"3,982 cc",630 hp,250 km/h,3.2 sec,"$161,000",Petrol,4,900 Nm
4,AUDI,AUDI R8 Gt,V10,"5,204 cc",602 hp,320 km/h,3.6 sec,"$253,290",Petrol,2,560 Nm


## Null Values in Dataset

In [5]:
df.isna().sum()

Company Names                0
Cars Names                   0
Engines                      0
CC/Battery Capacity          3
HorsePower                   0
Total Speed                  0
Performance(0 - 100 )KM/H    6
Cars Prices                  0
Fuel Types                   0
Seats                        0
Torque                       1
dtype: int64

## Dropping null value

In [6]:
df = df.dropna()

In [7]:
df.isna().sum()

Company Names                0
Cars Names                   0
Engines                      0
CC/Battery Capacity          0
HorsePower                   0
Total Speed                  0
Performance(0 - 100 )KM/H    0
Cars Prices                  0
Fuel Types                   0
Seats                        0
Torque                       0
dtype: int64

## Cleaning Function

In [8]:
def parse_value(value):
    if isinstance(value, str):
        # if "-" in value:
        #     # Split the range and calculate the average
        #     start, end = map(int, map(str.strip,value.split("-")))
        #     return (start + end) / 2
        # elif "/" in value:
        #     # Split the range and calculate the average
        #     start, end = map(float, map(str.strip,value.split("/")))
        #     return (start + end) / 2
        if len(re.findall('[\d.]+',value))>1:
            # Return the single value as a float
            return sum(map(float,re.findall('[\d.]+',value)))/len(re.findall('[\d.]+',value))
        elif re.findall('[\d.]+',value): 
            return float(re.findall('[\d.]+',value)[0])
        elif value == '-': return None
        else: return value

## Cleaning all required columns

In [9]:
df.rename(columns={"Total Speed": "Total Speed (km/h)"}, inplace=True)
df["Total Speed (km/h)"] = df["Total Speed (km/h)"].apply(lambda x: x.lower() if isinstance(x, str) else x)
df["Total Speed (km/h)"] = df["Total Speed (km/h)"].str.replace(" km/h", "", regex=False)

In [10]:
df.rename(columns={"Performance(0 - 100 )KM/H": "Performance(0 - 100 )KM/H (in sec)"}, inplace=True)
df["Performance(0 - 100 )KM/H (in sec)"] = df["Performance(0 - 100 )KM/H (in sec)"].str.replace(" sec", "", regex=False)

In [11]:
df.rename(columns={"CC/Battery Capacity": "Battery Capacity (cc)"}, inplace=True)
df["Battery Capacity (cc)"] = df["Battery Capacity (cc)"].str.replace(" cc", "", regex=False)
df["Battery Capacity (cc)"] = df["Battery Capacity (cc)"].str.replace(" kwh", "", regex=False)
df["Battery Capacity (cc)"] = df["Battery Capacity (cc)"].str.replace(",", "", regex=False)
df["Battery Capacity (cc)"] = df["Battery Capacity (cc)"].apply(parse_value)

In [12]:
df["HorsePower"] = df["HorsePower"].str.replace(" hp", "", regex=False)
df["HorsePower"] = df["HorsePower"].str.replace("~", "", regex=False)
df["HorsePower"] = df["HorsePower"].str.replace(" HP", "", regex=False)
df["HorsePower"] = df["HorsePower"].apply(parse_value)

In [13]:
df["Torque"] = df["Torque"].str.replace(" Nm", "", regex=False)
df["Torque"] = df["Torque"].apply(parse_value)

In [14]:
df.rename(columns={"Cars Prices": "Price ($)"}, inplace=True)
df["Price ($)"] = df["Price ($)"].str.replace("$", "", regex=False)
df["Price ($)"] = df["Price ($)"].str.replace(",", "", regex=False)
df["Price ($)"] = df["Price ($)"].apply(parse_value)

In [15]:
df["Seats"] = df["Seats"].apply(parse_value)

In [16]:
df["Fuel Types"].unique()

array(['plug in hyrbrid', 'Petrol', 'Diesel', 'Hybrid', 'Electric',
       'Petrol/Diesel', 'Plug-in Hybrid', 'Petrol/AWD', 'Petrol/Hybrid',
       'Hydrogen', 'Diesel/Petrol', 'Petrol/EV', 'Hybrid/Electric',
       'Petrol, Hybrid', 'Petrol, Diesel', 'Hybrid (Petrol)',
       'CNG/Petrol', 'Hybrid/Petrol'], dtype=object)

In [17]:
fuel_mapping = {
    'plug in hyrbrid': 'Plug-in Hybrid',
    'Plug-in Hybrid': 'Plug-in Hybrid',
    'Hybrid/Electric': 'Hybrid',
    'Hybrid (Petrol)': 'Hybrid',
    'Petrol/Hybrid': 'Hybrid',
    'Hybrid/Petrol': 'Hybrid',
    'Petrol, Hybrid': 'Hybrid',
    'Petrol, Diesel': 'Petrol/Diesel',
    'Diesel/Petrol': 'Petrol/Diesel',
    'Petrol/AWD': 'Petrol',
    'Petrol/EV': 'Electric',
    'CNG/Petrol': 'CNG',
    # Keep others as is (e.g., 'Petrol', 'Diesel', 'Electric', 'Hydrogen')
}

# Apply the mapping
df["Fuel Types"] = df["Fuel Types"].replace(fuel_mapping)

In [18]:
df.head(5)

,Company Names,Cars Names,Engines,Battery Capacity (cc),HorsePower,Total Speed (km/h),Performance(0 - 100 )KM/H (in sec),Price ($),Fuel Types,Seats,Torque
0,FERRARI,SF90 STRADALE,V8,3990.0,963.0,340,2.5,1100000.0,Plug-in Hybrid,2.0,800.0
1,ROLLS ROYCE,PHANTOM,V12,6749.0,563.0,250,5.3,460000.0,Petrol,5.0,900.0
2,Ford,KA+,1.2L Petrol,1200.0,77.5,165,10.5,13500.0,Petrol,5.0,120.0
3,MERCEDES,GT 63 S,V8,3982.0,630.0,250,3.2,161000.0,Petrol,4.0,900.0
4,AUDI,AUDI R8 Gt,V10,5204.0,602.0,320,3.6,253290.0,Petrol,2.0,560.0


In [19]:
df = df.dropna()

In [20]:
df.shape

(1203, 11)

## Saving the cleaned dataset

In [21]:
print("Saving Cleaned Dataset") 
df.to_csv('car_dataset.csv')

Saving Cleaned Dataset
